In [2]:
import pandas as pd
import numpy as np

from scipy.stats import shapiro, levene, f_oneway
import scikit_posthocs as sp

import matplotlib.pyplot as plt

df = pd.read_csv('../static/result.csv')
df.head()

length = [*range(1,25)]

replacement_ans = {
    "Agree": 1,
    "Unsure": 0,
    "Disagree": -1,
}

replacement_conf = {
    "7-Extremely confident": 7,
    "6-Very confident": 6,
    "5-Somewhat confident": 5,
    "4-Neutral": 4,
    "3-Somewhat lacking confidence": 3,
    "2-Very lacking confidence": 2,
    "1-Extremely lacking confidence": 1,
}

condition = [
    "control",
    "ai-edited-image",
    "ai-edited-video",
    "ai-edited-image-video",
]

induced_imgs = [1,2,3,4,9,10,11,12,17,18,19,20]
non_induced_imgs = [5,6,7,8,13,14,15,16,21,22,23,24]

correct_is_true = [6,7,9,10,14,18,21,23]
correct_is_false = [1,2,3,4,5,8,11,12,13,15,16,17,19,20,22,24]

# get filtered columns
filtered_cols = [[f"{i}_false_memories_ans", f"{i}_false_memories_conf"] for i in length]
filtered_cols = [i for sublist in filtered_cols for i in sublist]

# remove headers and replace values
head = df.head(1)
df = df[2:][['condition', *filtered_cols]]
df.replace(replacement_ans, inplace=True)
df.replace(replacement_conf, inplace=True)

result = df.copy()
result = result[['condition'] + [f"{i}_false_memories_ans" for i in length]  + [f"{i}_false_memories_conf" for i in length]]

# convert raw answer to score (1 -> correct, 0 -> unsure, -1 -> wrong)
for i in length:
    multiplier = 1 if i in correct_is_true else -1
    result[f"{i}_false_memories_ans"] *= multiplier

/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_2477/478375063.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_ans, inplace=True)
/var/folders/bw/64y3yr9n6v50fgm86wtx236c0000gn/T/ipykernel_2477/478375063.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replacement_conf, inplace=True)


In [4]:
# Generate Raw Tables

def summarize(cat, df):
    # get sum
    df['overall'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25)]]].sum(axis=1)
    df['induced'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25)] if i in induced_imgs]].sum(axis=1)
    df['non induced'] = df[[f"{i}_false_memories_ans" for i in [*range(1,25) ] if i in non_induced_imgs]].sum(axis=1)

    # get percentages in % in two decimals
    df['overall %'] = (df['overall'] / 24) * 100
    df['induced %'] = (df['induced'] / 12) * 100
    df['non induced %'] = (df['non induced'] / 12) * 100

    # filter
    df = df[['condition', 'overall', 'overall %', 'induced', 'induced %', 'non induced', 'non induced %']]

    print(cat)
    display(df.groupby('condition').mean().round(2))
    
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)
unsure_memories = result.copy().replace(1,-1).replace(0,1).replace(-1,0)

print("AMOUNT")
summarize('Non False Memories', true_memories)
summarize('Unsure Memories', unsure_memories)
summarize('False Memories', false_memories)

def confidence(cat, result, raw):
    print(cat)
    # display(result)
    # display(raw[['condition', *[f"{i}_false_memories_conf" for i in length]]])
    
    # get confidence: for each row {i}_confidence = result[{i}_false_memories_ans] * raw[{i}_false_memories_conf]
    for i in length:
        result[f"{i}_conf"] = result[f"{i}_false_memories_ans"] * raw[f"{i}_false_memories_conf"]
    
    result['overall'] = result[[f"{i}_conf" for i in length]].sum(axis=1) / 14
    result['induced'] = result[[f"{i}_conf" for i in length if i in induced_imgs]].sum(axis=1) / 7
    result['non induced'] = result[[f"{i}_conf" for i in length if i in non_induced_imgs]].sum(axis=1) / 7

    result = result[['condition', 'overall', 'induced', 'non induced']]
    display(result.groupby('condition').mean().round(2))
        
true_memories = result.copy().replace(-1,0)
false_memories = result.copy().replace(1,0).replace(-1,1)
unsure_memories = result.copy().replace(1,-1).replace(0,1).replace(-1,0)

print("\nCONFIDENCE")
confidence("Non False Memories", true_memories, df)
confidence("Unsure Memories", unsure_memories, df)
confidence("False Memories", false_memories, df)

AMOUNT
Non False Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
ai-edited-image,13.65,56.86,6.47,53.92,7.18,59.80
ai-edited-image-video,12.10,50.42,5.26,43.83,6.84,57.00
ai-edited-video,13.82,57.58,6.98,58.17,6.84,57.00
control,15.06,62.76,7.94,66.16,7.12,59.35


Unsure Memories


,overall,overall %,induced,induced %,non induced,non induced %
condition,,,,,,
ai-edited-image,3.61,15.03,1.76,14.71,1.84,15.36
ai-edited-image-video,4.58,19.08,2.10,17.50,2.48,20.67
ai-edited-video,4.28,17.83,2.18,18.17,2.10,17.50
control,3.96,16.50,1.80,14.97,2.16,18.03


False Memories


...

,overall,induced,non induced
condition,,,
ai-edited-image,5.21,4.96,5.47
ai-edited-image-video,4.95,4.23,5.67
ai-edited-video,5.64,5.71,5.56
control,5.98,6.32,5.64


Unsure Memories


,overall,induced,non induced
condition,,,
ai-edited-image,0.96,0.93,0.98
ai-edited-image-video,1.26,1.13,1.40
ai-edited-video,1.21,1.21,1.20
control,1.08,1.01,1.16


False Memories


,overall,induced,non induced
condition,,,
ai-edited-image,2.42,2.70,2.13
ai-edited-image-video,2.94,3.80,2.09
ai-edited-video,2.22,2.18,2.27
control,1.86,1.74,1.97
